In [2]:
import warnings
warnings.filterwarnings("ignore")


In [5]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
sns.set_style("whitegrid")
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
train=pd.read_csv('....../Walmart/train.csv')
test=pd.read_csv('......./Walmart/test.csv')
store=pd.read_csv('....../Walmart/stores.csv')
feature=pd.read_csv('...../Walmart/features.csv')

FileNotFoundError: [Errno 2] No such file or directory: '....../Walmart/train.csv'

In [4]:
train.head()

NameError: name 'train' is not defined

In [ ]:
feature.head()

In [ ]:
merge_df=pd.merge(train,feature, on=['Store','Date'], how='inner')

In [ ]:
merge_df.head()

In [ ]:
merge_df.describe().transpose()


In [ ]:
from datetime import datetime as dt

In [ ]:
merge_df['DateTimeObj']=[dt.strptime(x,'%Y-%m-%d') for x in list(merge_df['Date'])]
merge_df['DateTimeObj'].head()

In [ ]:
plt.plot(merge_df[(merge_df.Store==1)].DateTimeObj, merge_df[(merge_df.Store==1)].Weekly_Sales, 'ro')
plt.show()

In [ ]:
weeklysales=merge_df.groupby(['Store','Date'])['Weekly_Sales'].apply(lambda x:np.sum(x))
weeklysales[0:5]

In [ ]:
weeklysaledept=merge_df.groupby(['Store','Dept'])['Weekly_Sales'].apply(lambda x:np.sum(x))
weeklysaledept[0:5]

In [ ]:
weeklyscale=weeklysales.reset_index()
weeklyscale[0:5]

In [ ]:
walmartstore=pd.merge(weeklyscale, feature, on=['Store', 'Date'], how='inner')
walmartstore.head()

In [ ]:
walmartstoredf = walmartstore.iloc[:, list(range(5)) + list(range(10,13))]

In [ ]:
walmartstoredf.head()

In [ ]:
walmartstoredf['DateTimeObj'] = [dt.strptime(x, '%Y-%m-%d') for x in list(walmartstoredf['Date'])]
weekNo=walmartstoredf.reset_index()

In [ ]:
weekNo = [(x - walmartstoredf['DateTimeObj'][0]) for x in list(walmartstoredf['DateTimeObj'])]

In [ ]:
walmartstoredf['Week'] = [np.timedelta64(x, 'D').astype(int)/7 for x in weekNo]

In [ ]:
walmartstoredf.head()

In [ ]:
plt.plot(walmartstoredf.DateTimeObj, walmartstoredf.Weekly_Sales, 'ro')
plt.show()

In [ ]:
walmartstoredf['IsHolidayInt'] = [int(x) for x in list(walmartstoredf.IsHoliday)]

In [ ]:
walmartstoredf.head()

In [ ]:
walmartstoredf.Store.unique()

In [ ]:
train_WM, test_WM = train_test_split(walmartstoredf, test_size=0.3,random_state=42)

In [ ]:
plt.plot(walmartstoredf[(walmartstoredf.Store==1)].Week, walmartstoredf[(walmartstoredf.Store==1)].Weekly_Sales, 'ro')
plt.show()

In [ ]:
XTrain = train_WM[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Week', 'IsHolidayInt']]
YTrain = train_WM['Weekly_Sales']

In [ ]:
XTest = test_WM[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Week', 'IsHolidayInt']]
YTest = test_WM['Weekly_Sales']

In [ ]:
wmLinear = linear_model.LinearRegression(normalize=True)
wmLinear.fit(XTrain, YTrain)

In [ ]:
wmLinear.coef_

In [ ]:
#Performance on the test data sets
YHatTest = wmLinear.predict(XTest)

In [ ]:
plt.plot(YTest, YHatTest,'ro')
plt.plot(YTest, YTest,'b-')
plt.show()

In [ ]:
walmartstoredf['Store'].unique()

In [ ]:
Store_Dummies = pd.get_dummies(walmartstoredf.Store, prefix='Store').iloc[:,1:]
walmartstoredf = pd.concat([walmartstoredf, Store_Dummies], axis=1)

In [ ]:
walmartstoredf.head()

In [ ]:
train_WM, test_WM = train_test_split(walmartstoredf, test_size=0.3,random_state=42)
XTrain = train_WM.iloc[:,([3,4,5,6] + [9,10]) + list(range(11,walmartstoredf.shape[1]))]
yTrain = train_WM.Weekly_Sales
                                                    
XTest = test_WM.iloc[:,([3,4,5,6] + [9,10]) + list(range(11,walmartstoredf.shape[1]))]
yTest=test_WM.Weekly_Sales

In [ ]:
XTrain.head()

In [ ]:
wmLinear = linear_model.LinearRegression(normalize=True)
wmLinear.fit(XTrain, YTrain)

In [ ]:
#Performance on the test data sets
YHatTest = wmLinear.predict(XTest)
plt.plot(YTest, YHatTest,'ro')
plt.plot(YTest, YTest,'b-')
plt.show()

In [ ]:
# calculate the accuray of the model by sum of Square and mean absolute prediction error
MAPE = np.mean(abs((YTest - YHatTest)/YTest))
MSSE = np.mean(np.square(YHatTest - YTest))

print(MAPE, MSSE)

In [ ]:
# Dimensionality Reduction

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
alphas = np.linspace(10, 20, 10)

In [ ]:
testError = np.empty(10)

for i, alpha in enumerate(alphas) :
    
    lasso = Lasso(alpha=alpha)
    lasso.fit(XTrain, YTrain)
    testError[i] = mean_squared_error(YTest, lasso.predict(XTest))


In [ ]:
plt.plot(alphas, testError, 'r-')
plt.show()

In [ ]:
wmLinear = linear_model.LinearRegression(normalize=True)
wmLinear

In [ ]:
lasso = Lasso(alpha=17)
lasso.fit(XTrain, YTrain)